In [1]:
import torch
import numpy as np
import torch.nn.functional as F
from scipy.spatial.distance import directed_hausdorff

def compute_hausdorff_distance(mask1, mask2):
    """
    计算两个掩码之间的豪斯多夫距离。
    
    参数:
    mask1 (torch.Tensor): 第一个掩码，形状为 (H, W) 或 (D, H, W)。
    mask2 (torch.Tensor): 第二个掩码，形状为 (H, W) 或 (D, H, W)。
    
    返回:
    float: 豪斯多夫距离。
    """
    # 将掩码转换为点集
    points1 = torch.nonzero(mask1).float()  # 获取非零元素的坐标
    points2 = torch.nonzero(mask2).float()  # 获取非零元素的坐标
    
    # 将点集转换为 NumPy 数组，因为 scipy 的 directed_hausdorff 需要 NumPy 数组
    points1_np = points1.cpu().numpy()
    points2_np = points2.cpu().numpy()
    
    # 计算双向豪斯多夫距离
    hd1 = directed_hausdorff(points1_np, points2_np)[0]
    hd2 = directed_hausdorff(points2_np, points1_np)[0]
    
    # 取最大值作为豪斯多夫距离
    hd = max(hd1, hd2)
    
    return hd

def compute_h95(mask1, mask2):
    """
    计算两个掩码之间的 H95 指标。
    
    参数:
    mask1 (torch.Tensor): 第一个掩码，形状为 (H, W) 或 (D, H, W)。
    mask2 (torch.Tensor): 第二个掩码，形状为 (H, W) 或 (D, H, W)。
    
    返回:
    float: H95 指标。
    """
    # 计算豪斯多夫距离
    hd = compute_hausdorff_distance(mask1, mask2)
    
    # 计算 95 百分位数
    h95 = np.percentile(hd, 95)
    
    return h95

In [2]:
# from tqdm import tqdm
# # 测试数据
# batch_size = 2
# num_classes = 4
# spatial_dims = (128, 128, 128)

# # 生成随机标签和预测张量
# labels = torch.randint(0, 2, (batch_size, num_classes, *spatial_dims)).float()  # 标签
# predictions = torch.randint(0, 2, (batch_size, num_classes, *spatial_dims)).float()  # 预测

# # 计算每个样本和每个类别的 H95 指标
# h95_results = torch.zeros(batch_size, num_classes)  # 存储 H95 结果

# for i in tqdm(range(batch_size)):  # 遍历每个样本
#     for j in tqdm(range(num_classes)):  # 遍历每个类别
#         label_mask = labels[i, j]  # 获取当前样本和类别的标签掩码
#         pred_mask = predictions[i, j]  # 获取当前样本和类别的预测掩码
        
#         # 计算 H95 指标
#         h95 = compute_h95(label_mask, pred_mask)
#         h95_results[i, j] = h95

# print("H95 Results for each sample and class:")
# print(h95_results)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pred = torch.rand([2, 4, 128, 128, 128]).float().to(device=device)
mask = torch.randint(0, 4, [2, 128, 128, 128])
mask = F.one_hot(mask).permute(0, 4, 1, 2, 3).float().to(device=device)

pred.shape, mask.shape


(torch.Size([2, 4, 128, 128, 128]), torch.Size([2, 4, 128, 128, 128]))

In [4]:
# compute_h95(pred, mask)

In [11]:
distance = torch.cdist(pred.view(2, -1, 4), mask.reshape(2, -1, 4).shape)
# min_distance_pred_to_mask = torch.min(distance, dim=1)[0]


RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [16]:
mask.reshape(2, -1, 4).shape

torch.Size([2, 2097152, 4])